# Title
[]()

# Create classes

In [5]:
from with_sql_session import remote_sql_session
import secrets
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid


# def bulk_save_objects(new_rows, engine=None):
#     @remote_sql_session
#     def save_objects(session):
#         session.bulk_save_objects(new_rows)
#         session.commit()

#     return save_objects


Base = declarative_base()

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

new_rows = [
    Article(
        id="1", title="Test Article", 
        publication="Test Journal", text_body="This is a test text", uuid=str(uuid.uuid4())
        )
    ]

# save_objects = bulk_save_objects(new_rows)
# save_objects()

# Get table

## 1.1 using .query

In [8]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd


@remote_sql_session
def get_table(session, table='articles', filter=None):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.query(Article).all()
    # df = pd.DataFrame(q)
    # # df = pd.DataFrame(q.fetchall())
    # return df
    return q


table = get_table()

In [12]:
vars(table[0])

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1cb95f9e410>,
 'id': 1,
 'text_body': 'This is a test text',
 'title': 'Test Article',
 'uuid': 'a678c60d-f4a6-4727-bbaa-cc2cc958f52d',
 'publication': 'Test Journal'}

## 1.2 using .execute

In [13]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd


@remote_sql_session
def get_table(session, query='SELECT *', table='articles'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df


get_table()

,id,title,publication,text_body,uuid
0,1,Test Article,Test Journal,This is a test text,a678c60d-f4a6-4727-bbaa-cc2cc958f52d


# Update tables

In [3]:

from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

In [8]:
get_table()

NameError: name 'get_table' is not defined

## 2.0

In [10]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd

def bulk_save_objects(new_rows, engine=None):
    @remote_sql_session
    def save_objects(session):
        session.bulk_save_objects(new_rows)
        session.commit()

    return save_objects


@remote_sql_session
def get_table(session, query='SELECT *', table='articles'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

new_rows = [
    Publications(
        id=1, name="PLOS One"
        )
    ]

# save_objects = bulk_save_objects(new_rows)
# save_objects()

In [12]:
get_table(table='publications')

,id,name
0,1,PLOS One


## 2.1

In [14]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd
from sqlalchemy.dialects.postgresql import insert

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        statement = insert(Publications).values(new_rows)
        session.execute(statement)
        session.commit()
        print('Rows inserted:')
        print([f"\t{row['name']}" for row in new_rows])

    return insert_rows

values = [
    {'id': 2, 'name': 'BMJ Open'},
]
bulk_insert(values)()

get_table()

Rows inserted:
['\tBMJ Open']


,id,name
0,1,PLOS One
1,2,BMJ Open


# 3 Update tables

In [15]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
# from sqlalchemy.dialects.postgresql import UUID
# import uuid
import pandas as pd
from sqlalchemy.dialects.postgresql import insert


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        statement = insert(Publications).values(new_rows)
        session.execute(statement)
        session.commit()
        print('Rows inserted:')
        print([f"\t{row['name']}" for row in new_rows])

    return insert_rows

values = [
    {'name': 'PLOS Medicine'},
]
bulk_insert(values)()

get_table()

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "id" of relation "publications" violates not-null constraint
DETAIL:  Failing row contains (null, PLOS Medicine).

[SQL: INSERT INTO publications (name) VALUES (%(name_m0)s)]
[parameters: {'name_m0': 'PLOS Medicine'}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## 3.1 Able to insert rows using .add

In [2]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
from sqlalchemy.dialects.postgresql import insert


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

values = [
    Publications(name='PLOS Medicine'),
]
bulk_insert(values)()

get_table()

Rows inserted:


,id,name
0,1,PLOS Medicine


In [3]:
values = [
    Publications(name='PLOS One'),
]
bulk_insert(values)()

get_table()

Rows inserted:


,id,name
0,1,PLOS Medicine
1,2,PLOS One


# Iteration 4: Relate tables to each other.

In [1]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

values = [
    Publications(name='PLOS Medicine'),
]
bulk_insert(values)()

get_table()


NoForeignKeysError: Could not determine join condition between parent/child tables on relationship Publications.articles - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

## 4.1

In [3]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    # articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    # publication = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    # publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
article_list = [
    Article(
        title='Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women',
        ),
]
bulk_insert(article_list)()
# get_table()
get_table(table='articles')


Rows inserted:


,id,title
0,1,Hypohydration but not Menstrual Phase Influenc...


## 4.11

In [2]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    # articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    # publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
article_list = [
    Article(
        title='Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women',
        ),
]
# bulk_insert(article_list)()
# get_table()
get_table(table='articles')


,id,title,publication_id
0,1,Hypohydration but not Menstrual Phase Influenc...,3


## 4.111 got it

In [4]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
article_list = [
    Article(
        title='Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women',
        ),
]
# bulk_insert(article_list)()
# get_table()
get_table(table='articles')


,id,title,publication_id
0,1,Hypohydration but not Menstrual Phase Influenc...,3


In [5]:
get_table()

,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None


## 4.2

In [9]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
article_list = [
    Article(
        title='test PLOS One article'
        ),
]
# bulk_insert(article_list)()
get_table()
# get_table(table='articles')


Rows inserted:


,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None


In [10]:
get_table(table='articles')

,id,title,publication_id
0,1,Hypohydration but not Menstrual Phase Influenc...,3.0
1,2,test PLOS One article,NaN


In [11]:
article_list = [
    Article(
        title='test PLOS Medicine article', publication_id=1
        ),
]
bulk_insert(article_list)()
get_table()

Rows inserted:


,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None


In [12]:
get_table(table='articles')

,id,title,publication_id
0,1,Hypohydration but not Menstrual Phase Influenc...,3.0
1,2,test PLOS One article,NaN
2,3,test PLOS Medicine article,1.0


## 4.3 Create instances

In [14]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]

Journal_inst = Publications(name='International Journal of Obesity')
Journal_inst.articles = []

article_list = [
    Article(
        title='Weight stigma and health behaviors: evidence from the Eating in America Study'
        ),
]
# bulk_insert(article_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# get_table(table='articles')


AttributeError: 'list' object has no attribute '_sa_instance_state'

## 4.31

In [33]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255), ForeignKey('publications.name'))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            print('this: ', Journal_inst.articles)
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            # # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]

Journal_inst = Publications(name='International Journal of Obesity')
# Journal_inst.articles.append(article_list[0])


# article_list = [
#     Article(
#         title='Weight stigma and health behaviors: evidence from the Eating in America Study'
#         ),
# ]
# # bulk_insert(article_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# # get_table(table='articles')


AmbiguousForeignKeysError: Could not determine join condition between parent/child tables on relationship Publications.articles - there are multiple foreign key paths linking the tables.  Specify the 'foreign_keys' argument, providing a list of those columns which should be counted as containing a foreign key reference to the parent table.

In [ ]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]

Journal_inst = Publications(name='International Journal of Obesity')
Journal_inst.articles = []

article_list = [
    Article(
        title='Weight stigma and health behaviors: evidence from the Eating in America Study'
        ),
]
# bulk_insert(article_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# get_table(table='articles')


AttributeError: 'list' object has no attribute '_sa_instance_state'

In [27]:
print(Journal_inst)
print(Journal_inst.articles)

## 4.32

In [34]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = Column(Integer)
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]

Journal_inst = Publications(name='International Journal of Obesity')
Journal_inst.articles = []

article_list = [
    Article(
        title='Weight stigma and health behaviors: evidence from the Eating in America Study'
        ),
]
# bulk_insert(article_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# get_table(table='articles')


NoForeignKeysError: Could not determine join condition between parent/child tables on relationship Publications.articles - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

## 4.33

In [42]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]

Journal_inst = Publications(name='International Journal of Obesity')

article_list = [
    Article(
        title='Weight stigma and health behaviors: evidence from the Eating in America Study',
        # publication_id=Journal_inst.id
        ),
]
# bulk_insert(article_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# get_table(table='articles')


Rows inserted:


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "articles" violates foreign key constraint "articles_fk"
DETAIL:  Key (id)=(4) is not present in table "publications".

[SQL: INSERT INTO articles (title, publication_id) VALUES (%(title)s, %(publication_id)s) RETURNING articles.id]
[parameters: {'title': 'Weight stigma and health behaviors: evidence from the Eating in America Study', 'publication_id': None}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [40]:

get_table(table='articles')

,id,title,publication_id
0,1,Hypohydration but not Menstrual Phase Influenc...,3.0
1,2,test PLOS One article,NaN
2,3,test PLOS Medicine article,1.0


## 4.34

In [45]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='International Journal of Obesity')
journal_list = [Journal_inst]

# article_list = [
#     Article(
#         title='Weight stigma and health behaviors: evidence from the Eating in America Study',
#         # publication_id=Journal_inst.id
#         ),
# ]
bulk_insert(journal_list)()
# bulk_append(article_list, Journal_inst)()
# get_table()
# # get_table(table='articles')


Rows inserted:


In [50]:
get_table()

,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None
3,4,International Journal of Obesity,None


In [48]:
Journal_inst = Publications(name='International Journal of Obesity')

article_list = [
    Article(
        title='Weight stigma and health behaviors: evidence from the Eating in America Study',
        # publication_id=Journal_inst.id
        ),
]
# bulk_insert(journal_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# # get_table(table='articles')

Rows inserted:


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "articles" violates foreign key constraint "articles_fk"
DETAIL:  Key (id)=(5) is not present in table "publications".

[SQL: INSERT INTO articles (title, publication_id) VALUES (%(title)s, %(publication_id)s) RETURNING articles.id]
[parameters: {'title': 'Weight stigma and health behaviors: evidence from the Eating in America Study', 'publication_id': None}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## 4.35

In [9]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    # id = Column(Integer, primary_key=True)
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = Column(String(100))
    article_id = Column(Integer, ForeignKey('articles.id'))
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    publication_id = Column(String(100))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        session.add(Journal_inst)
        session.flush()  # Obtain the generated primary key value
        print(f'Publication ID: {Journal_inst.id}')
        print('Rows inserted:')

        for row in article_list:
            # article = Article(title=row, publication_id=Journal_inst.id)
            article = Article(title=row)

            session.add(article)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])

    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    # Article(
    #     title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
    #     ),
    'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()
bulk_append(article_list, Journal_inst)()
get_table()
# # get_table(table='articles')


Publication ID: 21
Rows inserted:


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "articles" violates foreign key constraint "articles_fk"
DETAIL:  Key (id)=(16) is not present in table "publications".

[SQL: INSERT INTO articles (title, publication_id) VALUES (%(title)s, %(publication_id)s) RETURNING articles.id]
[parameters: {'title': 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory', 'publication_id': None}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## 4.4 Put the foreign key in Article class

In [1]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = Column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = Column(Integer)
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    # publication_id = Column(String(100))
    publication_id = Column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# def bulk_append(article_list, Journal_inst, engine=None):
#     @remote_sql_session
#     def insert_rows(session):
#         session.add(Journal_inst)
#         session.flush()  # Obtain the generated primary key value
#         print(f'Publication ID: {Journal_inst.id}')
#         print('Rows inserted:')

#         for row in article_list:
#             # article = Article(title=row, publication_id=Journal_inst.id)
#             article = Article(title=row)

#             session.add(article)
#             session.commit()
#             # print([f"\t{row['name']}"])
#             print([f"\t{row}"])
#     return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            print('this: ', Journal_inst.articles)
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])
    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    Article(
        title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
        ),
    # 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()

bulk_append(article_list, Journal_inst)()
get_table()

# # get_table(table='articles')


Rows inserted:
this:  []


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "articles" violates foreign key constraint "articles_fk"
DETAIL:  Key (id)=(18) is not present in table "publications".

[SQL: INSERT INTO articles (title, publication_id) VALUES (%(title)s, %(publication_id)s) RETURNING articles.id]
[parameters: {'title': 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory', 'publication_id': 23}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## 4.41

In [6]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = Column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = Column(Integer)
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    # publication_id = Column(String(100))
    publication_id = Column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        session.add(Journal_inst)
        session.flush()  # Obtain the generated primary key value
        print(f'Publication ID: {Journal_inst.id}')
        print('Rows inserted:')
        session.commit()

        for row in article_list:
            article = Article(title=row, publication_id=publication_id)
            # article = Article(title=row)

            session.add(article)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])
    return insert_rows

# def bulk_append(article_list, Journal_inst, engine=None):
#     @remote_sql_session
#     def insert_rows(session):
#         print('Rows inserted:')

#         for row in article_list:
#             print('this: ', Journal_inst.articles)
#             Journal_inst.articles.append(row)
#             session.add(Journal_inst)
#             session.commit()
#             # print([f"\t{row['name']}"])
#             print([f"\t{row}"])
#     return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    # Article(
    #     title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
    #     ),
    'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()

bulk_append(article_list, Journal_inst)()
get_table()

# # get_table(table='articles')


Publication ID: 28
Rows inserted:


NameError: name 'publication_id' is not defined

In [7]:
get_table()

,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None
3,4,International Journal of Obesity,None
4,27,Physiology & Behavior,None
5,28,Physiology & Behavior,None


## 4.42 works if not committing

In [7]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = Column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = Column(Integer)
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    # publication_id = Column(String(100))
    publication_id = Column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# def bulk_append(article_list, Journal_inst, engine=None):
#     @remote_sql_session
#     def insert_rows(session):
#         session.add(Journal_inst)
#         session.flush()  # Obtain the generated primary key value
#         print(f'Publication ID: {Journal_inst.id}')
#         print('Rows inserted:')

#         for row in article_list:
#             article = Article(title=row, publication_id=Journal_inst.id)
#             # article = Article(title=row)
#             print('before add')
#             session.add(article)
#             print('this: ', Journal_inst.articles)
#             # session.commit()
#             # print([f"\t{row['name']}"])
#             print([f"\t{row}"])
#     return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')

        for row in article_list:
            print('this: ', Journal_inst.articles)
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            # session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])
            print(row.publication_id)
            print(Journal_inst.id)
    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    Article(
        title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
        ),
    # 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()

bulk_append(article_list, Journal_inst)()
get_table()

# # get_table(table='articles')


Rows inserted:
this:  []
['\t<__main__.Article object at 0x00000188A4819750>']
None
None


,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None
3,4,International Journal of Obesity,None
4,27,Physiology & Behavior,None
5,28,Physiology & Behavior,None


## 4.43 Still unable to link the tables

In [12]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = Column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = Column(Integer)
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    # publication_id = Column(String(100))
    publication_id = Column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# def bulk_append(article_list, Journal_inst, engine=None):
#     @remote_sql_session
#     def insert_rows(session):
#         session.add(Journal_inst)
#         session.flush()  # Obtain the generated primary key value
#         print(f'Publication ID: {Journal_inst.id}')
#         print('Rows inserted:')

#         for row in article_list:
#             article = Article(title=row, publication_id=Journal_inst.id)
#             # article = Article(title=row)
#             print('before add')
#             session.add(article)
#             print('this: ', Journal_inst.articles)
#             # session.commit()
#             # print([f"\t{row['name']}"])
#             print([f"\t{row}"])
#     return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        session.add(Journal_inst)
        session.flush()  # Obtain the generated primary key value
        print(f'Publication ID: {Journal_inst.id}')

        for row in article_list:
            print('this: ', Journal_inst.articles)
            Journal_inst.articles.append(row)
            row.publication_id = Journal_inst.id
            session.add(Journal_inst)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])
            print(row.publication_id)
    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    Article(
        title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
        ),
    # 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()

bulk_append(article_list, Journal_inst)()
get_table()

# # get_table(table='articles')


Rows inserted:
Publication ID: 41
this:  []
['\t<__main__.Article object at 0x00000188A4EDAC90>']
41


,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None
3,4,International Journal of Obesity,None
4,27,Physiology & Behavior,None
5,28,Physiology & Behavior,None
6,41,Physiology & Behavior,None


In [13]:
get_table(table='articles')

,id,title,publication_id
0,1,Hypohydration but not Menstrual Phase Influenc...,3.0
1,2,test PLOS One article,NaN
2,3,test PLOS Medicine article,1.0
3,27,"Food craving, cortisol and ghrelin responses i...",41.0


## 4.4 Update sqlalchemy package and Use mapped_column

In [1]:
from sqlalchemy.orm import mapped_column

In [2]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = mapped_column(Integer, primary_key=True)
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = mapped_column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = mapped_column(Integer)
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    # publication_id = Column(String(100))
    publication_id = mapped_column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# def bulk_append(article_list, Journal_inst, engine=None):
#     @remote_sql_session
#     def insert_rows(session):
#         session.add(Journal_inst)
#         session.flush()  # Obtain the generated primary key value
#         print(f'Publication ID: {Journal_inst.id}')
#         print('Rows inserted:')

#         for row in article_list:
#             article = Article(title=row, publication_id=Journal_inst.id)
#             # article = Article(title=row)
#             print('before add')
#             session.add(article)
#             print('this: ', Journal_inst.articles)
#             # session.commit()
#             # print([f"\t{row['name']}"])
#             print([f"\t{row}"])
#     return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        session.add(Journal_inst)
        session.flush()  # Obtain the generated primary key value
        print(f'Publication ID: {Journal_inst.id}')

        for row in article_list:
            print('this: ', Journal_inst.articles)
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])
            print(row.publication_id)
    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    Article(
        title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
        ),
    # 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()

# bulk_append(article_list, Journal_inst)()
# get_table()

# # get_table(table='articles')


c:\Users\silvh\.conda\envs\ginkgo\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
get_table()

ArgumentError: Textual SQL expression 'SELECT * from publication...' should be explicitly declared as text('SELECT * from publication...')

## 4.41

In [6]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Text, text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = mapped_column(Integer, primary_key=True)
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    name = mapped_column(String(100))
    # article_id = Column(Integer, ForeignKey('articles.id'))
    article_id = mapped_column(Integer)
    articles = relationship('Article', back_populates='publication', uselist=True)

class Article(Base):
    __tablename__ = 'articles'
    # id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    # publication_id = Column(String(100))
    publication_id = mapped_column(String(100), ForeignKey('publications.id'))
    # text_body = Column(Text, nullable=True)
    # uuid = Column(UUID, nullable=True)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(text(f'{query} from {table}'))
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

# def bulk_append(article_list, Journal_inst, engine=None):
#     @remote_sql_session
#     def insert_rows(session):
#         session.add(Journal_inst)
#         session.flush()  # Obtain the generated primary key value
#         print(f'Publication ID: {Journal_inst.id}')
#         print('Rows inserted:')

#         for row in article_list:
#             article = Article(title=row, publication_id=Journal_inst.id)
#             # article = Article(title=row)
#             print('before add')
#             session.add(article)
#             print('this: ', Journal_inst.articles)
#             # session.commit()
#             # print([f"\t{row['name']}"])
#             print([f"\t{row}"])
#     return insert_rows

def bulk_append(article_list, Journal_inst, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        session.add(Journal_inst)
        session.flush()  # Obtain the generated primary key value
        print(f'Publication ID: {Journal_inst.id}')

        for row in article_list:
            print('this: ', Journal_inst.articles)
            Journal_inst.articles.append(row)
            session.add(Journal_inst)
            session.commit()
            # print([f"\t{row['name']}"])
            print([f"\t{row}"])
            print(row.publication_id)
    return insert_rows

# values = [
#     Publications(
#         name='Journal of Applied Physiology',
#         ),
# ]
Journal_inst = Publications(name='Physiology & Behavior')
journal_list = [Journal_inst]

article_list = [
    # Article(
    #     title='Weight stigma and health behaviors: evidence from the Eating in America Study',
    #     # publication_id=Journal_inst.id
    #     ),
    Article(
        title='Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory',
        ),
    # 'Food craving, cortisol and ghrelin responses in modeling highly palatable snack intake in the laboratory'
]
# bulk_insert(journal_list)()

# bulk_append(article_list, Journal_inst)()
get_table()

# # get_table(table='articles')


,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None
3,4,International Journal of Obesity,None
4,27,Physiology & Behavior,None
5,28,Physiology & Behavior,None
6,41,Physiology & Behavior,None


In [7]:
bulk_append(article_list, Journal_inst)()
get_table()

Rows inserted:
Publication ID: 42
this:  []
['\t<__main__.Article object at 0x0000020BAD9B9FD0>']
42


,id,name,article_id
0,1,PLOS Medicine,None
1,2,PLOS One,None
2,3,Journal of Applied Physiology,None
3,4,International Journal of Obesity,None
4,27,Physiology & Behavior,None
5,28,Physiology & Behavior,None
6,41,Physiology & Behavior,None
7,42,Physiology & Behavior,None


# *End of Page*